In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.stats.diagnostic as diag
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [ ]:
# Global Variables
seed = random.seed(123)
number_clusters = 3

In [ ]:
# Variable classification
train_data = pd.read_csv('./data/train.csv', encoding = "ISO-8859-1")
test_data = pd.read_csv('./data/test.csv', encoding = "ISO-8859-1")
variables = pd.read_csv('./data/variables.txt', encoding = "ISO-8859-1")
quant_vars = list(variables.loc[(variables['Clasification'] == 'Cuantitativa')]['Variable'].values)
quali_vars = list(variables.loc[(variables['Clasification'] == 'Cualitativa')]['Variable'].values)[1:]

In [ ]:
def categorize(row): 
    if row['SalePrice'] > 0 and row['SalePrice'] <= 179280:
        return 'Low'
    elif row['SalePrice'] > 179280 and row['SalePrice'] < 326100:
        return 'Medium'
    else:
        return 'Expensive'

train_data['SalesCategories'] = train_data.apply(lambda row: categorize(row), axis=1)

In [ ]:
copied_train_data = train_data.copy()
copied_train_data = copied_train_data.fillna(0)

target = copied_train_data.pop('SalesCategories')
data = copied_train_data

categoricas = copied_train_data[quali_vars]
numericas = copied_train_data[quant_vars]

In [ ]:
#Se transforman las colunas usando los preprocesadores
numeric_preprocessor = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

# Se preparan los preprocesadores
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")


preprocesador = ColumnTransformer([
    ('one_hot_encoder', categorical_preprocessor, quali_vars),
    ('numerico', numeric_preprocessor, quant_vars)
],remainder="passthrough")

In [ ]:
for item in data:
  if (data[item].dtype == 'object'):
    data = data.astype({item: str})
X_train, X_test, y_train, y_test = train_test_split(data, target,test_size=0.3,train_size=0.7, random_state=42)

In [ ]:
modelo = make_pipeline(preprocesador, SVC(kernel="linear"))
modelo.fit(X_train,y_train)
print(modelo.score(X_test, y_test)) 
X_pred = modelo.predict(X_test)
print ("Accuracy:",metrics.accuracy_score(y_test, X_pred))
print ("Precision:", metrics.precision_score(y_test, X_pred,average='weighted') )
print ("Recall: ", metrics.recall_score(y_test, X_pred,average='weighted'))
print(confusion_matrix(X_pred, y_test))

In [ ]:
modelo = make_pipeline(preprocesador, SVC(kernel="poly", C=32, degree=4))
modelo.fit(X_train,y_train)
print(modelo.score(X_test, y_test))
X_pred = modelo.predict(X_test)
print ("Accuracy:",metrics.accuracy_score(y_test, X_pred))
print ("Precision:", metrics.precision_score(y_test, X_pred,average='weighted') )
print ("Recall: ", metrics.recall_score(y_test, X_pred,average='weighted'))
print(confusion_matrix(X_pred, y_test))

In [ ]:
modelo = make_pipeline(preprocesador, SVC(kernel="rbf", C=32, degree=2e-05))
modelo.fit(X_train,y_train)
print(modelo.score(X_test, y_test))
X_pred = modelo.predict(X_test)
print ("Accuracy:",metrics.accuracy_score(y_test, X_pred))
print ("Precision:", metrics.precision_score(y_test, X_pred,average='weighted') )
print ("Recall: ", metrics.recall_score(y_test, X_pred,average='weighted'))
print(confusion_matrix(X_pred, y_test))